<a href="https://colab.research.google.com/github/tu702019/Machine_Learning_Algorithm_and_Its_Application/blob/main/HW1_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
# 載入數據集
url_bankruptcy = 'https://raw.githubusercontent.com/tu702019/Machine_Learning_Algorithm_and_Its_Application/refs/heads/main/HW1_Data%20Preprocessing/bankruptcy(predict%20brankrupt%20or%20not).csv'
df_bankruptcy = pd.read_csv(url_bankruptcy)

url_diamonds = 'https://raw.githubusercontent.com/tu702019/Machine_Learning_Algorithm_and_Its_Application/refs/heads/main/HW1_Data%20Preprocessing/daimonds(predict%20price).csv'
df_diamonds = pd.read_csv(url_diamonds)

url_iris = 'https://raw.githubusercontent.com/tu702019/Machine_Learning_Algorithm_and_Its_Application/refs/heads/main/HW1_Data%20Preprocessing/iris_data.csv'
df_iris = pd.read_csv(url_iris)

url_stroke = 'https://raw.githubusercontent.com/tu702019/Machine_Learning_Algorithm_and_Its_Application/refs/heads/main/HW1_Data%20Preprocessing/stroke(predict%20stroke%20or%20not).csv'
df_stroke = pd.read_csv(url_stroke)

In [14]:
# 步驟1：探索數據集
print("Bankruptcy 數據集信息:")
print(f"數據集形狀: {df_bankruptcy.shape}")
print("前五行數據:")
print(df_bankruptcy.head())
print("\n缺失值統計:")
print(df_bankruptcy.isnull().sum().sum(), "個缺失值")
print(df_bankruptcy.isnull().sum()[df_bankruptcy.isnull().sum() > 0])

print("\nDiamonds 數據集信息:")
print(f"數據集形狀: {df_diamonds.shape}")
print("前五行數據:")
print(df_diamonds.head())
print("\n缺失值統計:")
print(df_diamonds.isnull().sum().sum(), "個缺失值")
print(df_diamonds.isnull().sum()[df_diamonds.isnull().sum() > 0])


Bankruptcy 數據集信息:
數據集形狀: (6819, 97)
前五行數據:
   Unnamed: 0  Bankrupt?  \
0           0          1   
1           1          1   
2           2          1   
3           3          1   
4           4          1   

    ROA(C) before interest and depreciation before interest  \
0                                           0.370594          
1                                           0.464291          
2                                           0.426071          
3                                           0.399844          
4                                           0.465022          

    ROA(A) before interest and % after tax  \
0                                 0.424389   
1                                 0.538214   
2                                 0.499019   
3                                 0.451265   
4                                 0.538432   

    ROA(B) before interest and depreciation after tax  \
0                                           0.405750    
1                 

In [15]:
# 步驟2：處理缺失值
# 2.1 處理bankruptcy數據集的缺失值
# 首先查看缺失值的具體情況
missing_values_bankruptcy = df_bankruptcy.isnull().sum()
columns_with_missing_bankruptcy = missing_values_bankruptcy[missing_values_bankruptcy > 0].index

# 針對不同類型的數據採用不同的填充策略
df_bankruptcy_filled = df_bankruptcy.copy()

for column in columns_with_missing_bankruptcy:
    # 判斷列的數據類型
    if df_bankruptcy[column].dtype == 'object':  # 分類變數
        # 用眾數填充
        mode_value = df_bankruptcy[column].mode()[0]
        df_bankruptcy_filled[column] = df_bankruptcy[column].fillna(mode_value)
    else:  # 數值型變數
        # 用中位數填充
        median_value = df_bankruptcy[column].median()
        df_bankruptcy_filled[column] = df_bankruptcy[column].fillna(median_value)

# 確認缺失值是否已填充
print("填充後的Bankruptcy數據集缺失值情況:", df_bankruptcy_filled.isnull().sum().sum())

# 2.2 處理diamonds數據集的缺失值
# 首先查看缺失值的具體情況
missing_values_diamonds = df_diamonds.isnull().sum()
columns_with_missing_diamonds = missing_values_diamonds[missing_values_diamonds > 0].index

# 針對不同類型的數據採用不同的填充策略
df_diamonds_filled = df_diamonds.copy()

for column in columns_with_missing_diamonds:
    # 判斷列的數據類型
    if df_diamonds[column].dtype == 'object':  # 分類變數
        # 用眾數填充
        mode_value = df_diamonds[column].mode()[0]
        df_diamonds_filled[column] = df_diamonds[column].fillna(mode_value)
    else:  # 數值型變數
        # 用中位數填充
        median_value = df_diamonds[column].median()
        df_diamonds_filled[column] = df_diamonds[column].fillna(median_value)

# 確認缺失值是否已填充
print("填充後的Diamonds數據集缺失值情況:", df_diamonds_filled.isnull().sum().sum())

填充後的Bankruptcy數據集缺失值情況: 0
填充後的Diamonds數據集缺失值情況: 0


In [16]:
# 步驟3：使用SMOTE處理數據不平衡（只適用於分類問題）
from imblearn.over_sampling import SMOTE

# 檢查Bankruptcy數據集的類別分布
print("Bankruptcy數據集類別分布:")
print(df_bankruptcy_filled["Bankrupt?"].value_counts())

# 準備特徵和標籤
X = df_bankruptcy_filled.drop(["Bankrupt?"], axis=1)
if "Unnamed: 0" in X.columns:
    X = X.drop("Unnamed: 0", axis=1)
y = df_bankruptcy_filled["Bankrupt?"]

print(f"原始數據集 - 非破產樣本數: {(y == 0).sum()}, 破產樣本數: {(y == 1).sum()}")

# 應用SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# 查看平衡後的類別分布
print("SMOTE後的類別分布:")
balanced_distribution = pd.Series(y_resampled).value_counts()
print(balanced_distribution)

# 創建平衡後的數據框
df_bankruptcy_balanced = pd.DataFrame(X_resampled, columns=X.columns)
df_bankruptcy_balanced["Bankrupt?"] = y_resampled


Bankruptcy數據集類別分布:
Bankrupt?
0    6599
1     220
Name: count, dtype: int64
原始數據集 - 非破產樣本數: 6599, 破產樣本數: 220
SMOTE後的類別分布:
Bankrupt?
1    6599
0    6599
Name: count, dtype: int64


In [20]:
# 步驟4：特徵選擇
from sklearn.feature_selection import SelectKBest, f_classif, f_regression
import warnings

# 暫時忽略特定警告
warnings.filterwarnings('ignore', category=UserWarning, message='Features .* are constant.')
warnings.filterwarnings('ignore', category=RuntimeWarning, message='invalid value encountered in divide')

# 4.1 Bankruptcy數據集的特徵選擇（分類問題）
# 去除目標變數，剩下的所有列都是特徵
X_bankruptcy = df_bankruptcy_balanced.drop("Bankrupt?", axis=1)
if "Unnamed: 0" in X_bankruptcy.columns:
    X_bankruptcy = X_bankruptcy.drop("Unnamed: 0", axis=1)  # 使用if檢查ID列是否存在
y_bankruptcy = df_bankruptcy_balanced["Bankrupt?"]  # 目標變數

original_features_bankruptcy = X_bankruptcy.shape[1]
print(f"Bankruptcy數據集原始特徵數量: {original_features_bankruptcy}")

# 使用ANOVA F-value進行特徵選擇
# 選擇前50%的特徵
k_features_bankruptcy = int(original_features_bankruptcy * 0.5)
selector_bankruptcy = SelectKBest(f_classif, k=k_features_bankruptcy)
X_bankruptcy_selected = selector_bankruptcy.fit_transform(X_bankruptcy, y_bankruptcy)

# 獲取選中的特徵索引和名稱
selected_features_mask_bankruptcy = selector_bankruptcy.get_support()
selected_features_bankruptcy = X_bankruptcy.columns[selected_features_mask_bankruptcy].tolist()

print(f"Bankruptcy數據集選擇後的特徵數量: {len(selected_features_bankruptcy)}")
print("選中的前10個特徵:", selected_features_bankruptcy[:10])

# 創建包含選定特徵的新DataFrame
df_bankruptcy_final = pd.DataFrame(X_bankruptcy_selected, columns=selected_features_bankruptcy)
df_bankruptcy_final["Bankrupt?"] = y_bankruptcy.values  # 添加目標變數

# 4.2 Diamonds數據集的特徵選擇（回歸問題）
# 去除目標變數和ID列
X_diamonds = df_diamonds_filled.drop("price", axis=1)
if "Unnamed: 0" in X_diamonds.columns:
    X_diamonds = X_diamonds.drop("Unnamed: 0", axis=1)  # 使用if檢查ID列是否存在
if "Unnamed: 0.1" in X_diamonds.columns:
    X_diamonds = X_diamonds.drop("Unnamed: 0.1", axis=1)  # 使用if檢查另一個ID列是否存在
y_diamonds = df_diamonds_filled["price"]  # 目標變數

original_features_diamonds = X_diamonds.shape[1]
print(f"Diamonds數據集原始特徵數量: {original_features_diamonds}")

# 處理分類特徵，轉換為數值
X_diamonds_encoded = pd.get_dummies(X_diamonds, drop_first=True)

encoded_features_count = X_diamonds_encoded.shape[1]
print(f"Diamonds數據集編碼後的特徵數量: {encoded_features_count}")

# 使用F回歸統計量進行特徵選擇
# 選擇前50%的特徵
k_features_diamonds = min(int(encoded_features_count * 0.5), encoded_features_count)
selector_diamonds = SelectKBest(f_regression, k=k_features_diamonds)
X_diamonds_selected = selector_diamonds.fit_transform(X_diamonds_encoded, y_diamonds)

# 獲取選中的特徵索引和名稱
selected_features_mask_diamonds = selector_diamonds.get_support()
selected_features_diamonds = X_diamonds_encoded.columns[selected_features_mask_diamonds].tolist()

print(f"Diamonds數據集選擇後的特徵數量: {len(selected_features_diamonds)}")
print("選中的前10個特徵:", selected_features_diamonds[:10])

# 創建最終的數據框
df_diamonds_encoded = pd.get_dummies(df_diamonds_filled.drop("price", axis=1), drop_first=True)
df_diamonds_final = pd.DataFrame(df_diamonds_encoded[selected_features_diamonds])
df_diamonds_final["price"] = y_diamonds.values  # 添加目標變數

Bankruptcy數據集原始特徵數量: 95
Bankruptcy數據集選擇後的特徵數量: 47
Diamonds數據集原始特徵數量: 9
Diamonds數據集編碼後的特徵數量: 23
Diamonds數據集選擇後的特徵數量: 11


In [22]:
# 步驟5：觀察特徵選擇對預測能力的影響
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier  # 用於Bankruptcy數據集
from sklearn.ensemble import RandomForestRegressor   # 用於Diamonds數據集
from sklearn.metrics import accuracy_score, r2_score

# 1. Bankruptcy數據集評估（分類問題）
print("評估Bankruptcy數據集的特徵選擇效果：")

# 準備原始特徵數據（不包含ID列，但包含所有其他特徵）
X_full = df_bankruptcy_balanced.drop(["Bankrupt?"], axis=1)
if "Unnamed: 0" in X_full.columns:
    X_full = X_full.drop("Unnamed: 0", axis=1)
y = df_bankruptcy_balanced["Bankrupt?"]

# 將數據分為訓練集和測試集
X_train_full, X_test_full, y_train, y_test = train_test_split(X_full, y, test_size=0.3, random_state=42)

# 使用RandomForest評估全部特徵的表現
rf_full = RandomForestClassifier(n_estimators=100, random_state=42)
# 使用交叉驗證評估
cv_scores_full = cross_val_score(rf_full, X_train_full, y_train, cv=5, scoring='accuracy')
print(f"使用全部特徵的平均準確率: {cv_scores_full.mean():.4f} ± {cv_scores_full.std():.4f}")

# 使用RandomForest評估選擇後特徵的表現
# 選擇的特徵是在步驟4中得到的
X_train_selected = X_train_full[selected_features_bankruptcy]  # 使用正確的變數名稱
X_test_selected = X_test_full[selected_features_bankruptcy]    # 使用正確的變數名稱

rf_selected = RandomForestClassifier(n_estimators=100, random_state=42)
cv_scores_selected = cross_val_score(rf_selected, X_train_selected, y_train, cv=5, scoring='accuracy')
print(f"使用選擇後特徵的平均準確率: {cv_scores_selected.mean():.4f} ± {cv_scores_selected.std():.4f}")

# 在測試集上評估
rf_full.fit(X_train_full, y_train)
rf_selected.fit(X_train_selected, y_train)

y_pred_full = rf_full.predict(X_test_full)
y_pred_selected = rf_selected.predict(X_test_selected)

test_acc_full = accuracy_score(y_test, y_pred_full)
test_acc_selected = accuracy_score(y_test, y_pred_selected)

print(f"測試集上全部特徵的準確率: {test_acc_full:.4f}")
print(f"測試集上選擇後特徵的準確率: {test_acc_selected:.4f}")
print(f"特徵數量減少了: {X_full.shape[1] - len(selected_features_bankruptcy)} 個 ({(X_full.shape[1] - len(selected_features_bankruptcy))/X_full.shape[1]*100:.1f}%)")

# 2. Diamonds數據集評估（回歸問題）
print("\n評估Diamonds數據集的特徵選擇效果：")

# 準備原始特徵數據
X_diamonds_full = df_diamonds_filled.drop(["price"], axis=1)
if "Unnamed: 0" in X_diamonds_full.columns:
    X_diamonds_full = X_diamonds_full.drop("Unnamed: 0", axis=1)
if "Unnamed: 0.1" in X_diamonds_full.columns:
    X_diamonds_full = X_diamonds_full.drop("Unnamed: 0.1", axis=1)
y_diamonds = df_diamonds_filled["price"]

# 處理分類特徵
X_diamonds_full_encoded = pd.get_dummies(X_diamonds_full, drop_first=True)

# 將數據分為訓練集和測試集
X_train_full, X_test_full, y_train, y_test = train_test_split(X_diamonds_full_encoded, y_diamonds, test_size=0.3, random_state=42)

# 使用RandomForest評估全部特徵的表現
rf_full = RandomForestRegressor(n_estimators=100, random_state=42)
cv_scores_full = cross_val_score(rf_full, X_train_full, y_train, cv=5, scoring='r2')
print(f"使用全部特徵的平均R²分數: {cv_scores_full.mean():.4f} ± {cv_scores_full.std():.4f}")

# 使用RandomForest評估選擇後特徵的表現
# 確保選擇的特徵存在於訓練集中
available_features = [f for f in selected_features_diamonds if f in X_train_full.columns]
X_train_selected = X_train_full[available_features]
X_test_selected = X_test_full[available_features]

rf_selected = RandomForestRegressor(n_estimators=100, random_state=42)
cv_scores_selected = cross_val_score(rf_selected, X_train_selected, y_train, cv=5, scoring='r2')
print(f"使用選擇後特徵的平均R²分數: {cv_scores_selected.mean():.4f} ± {cv_scores_selected.std():.4f}")

# 在測試集上評估
rf_full.fit(X_train_full, y_train)
rf_selected.fit(X_train_selected, y_train)

y_pred_full = rf_full.predict(X_test_full)
y_pred_selected = rf_selected.predict(X_test_selected)

test_r2_full = r2_score(y_test, y_pred_full)
test_r2_selected = r2_score(y_test, y_pred_selected)

print(f"測試集上全部特徵的R²分數: {test_r2_full:.4f}")
print(f"測試集上選擇後特徵的R²分數: {test_r2_selected:.4f}")
print(f"特徵數量減少了: {X_diamonds_full_encoded.shape[1] - len(available_features)} 個 ({(X_diamonds_full_encoded.shape[1] - len(available_features))/X_diamonds_full_encoded.shape[1]*100:.1f}%)")

# 簡單總結
print("\n特徵選擇效果總結：")
print(f"Bankruptcy數據集：準確率變化 {(test_acc_selected - test_acc_full) * 100:.2f}% 點")
print(f"Diamonds數據集：R²分數變化 {(test_r2_selected - test_r2_full) * 100:.2f}% 點")

評估Bankruptcy數據集的特徵選擇效果：
使用全部特徵的平均準確率: 0.9742 ± 0.0036
使用選擇後特徵的平均準確率: 0.9651 ± 0.0032
測試集上全部特徵的準確率: 0.9806
測試集上選擇後特徵的準確率: 0.9710
特徵數量減少了: 48 個 (50.5%)

評估Diamonds數據集的特徵選擇效果：
使用全部特徵的平均R²分數: 0.9455 ± 0.0034
使用選擇後特徵的平均R²分數: 0.8884 ± 0.0045
測試集上全部特徵的R²分數: 0.9469
測試集上選擇後特徵的R²分數: 0.8879
特徵數量減少了: 12 個 (52.2%)

特徵選擇效果總結：
Bankruptcy數據集：準確率變化 -0.96% 點
Diamonds數據集：R²分數變化 -5.91% 點


In [23]:
# 總結評估
if test_acc_selected >= test_acc_full and test_r2_selected >= test_r2_full:
    print("\n結論: 特徵選擇成功提高了兩個數據集的預測能力，同時減少了模型複雜度和訓練時間。")
elif test_acc_selected >= test_acc_full:
    print("\n結論: 特徵選擇提高了Bankruptcy數據集的預測能力，但對Diamonds數據集的預測能力有輕微影響。")
elif test_r2_selected >= test_r2_full:
    print("\n結論: 特徵選擇提高了Diamonds數據集的預測能力，但對Bankruptcy數據集的預測能力有輕微影響。")
else:
    print("\n結論: 特徵選擇雖然減少了模型複雜度和訓練時間，但對預測能力有輕微的負面影響。")
    print("然而，考慮到特徵數量的大幅減少，這種輕微的性能損失可能是可接受的權衡。")


結論: 特徵選擇雖然減少了模型複雜度和訓練時間，但對預測能力有輕微的負面影響。
然而，考慮到特徵數量的大幅減少，這種輕微的性能損失可能是可接受的權衡。


In [24]:
# 步驟6：保存處理後的數據集為.xlsx和.csv格式

# 保存Bankruptcy數據集
print("\n正在保存處理後的Bankruptcy數據集...")
df_bankruptcy_final.to_excel("bankruptcy_processed.xlsx", index=False)
df_bankruptcy_final.to_csv("bankruptcy_processed.csv", index=False)
print("Bankruptcy數據集已保存為 'bankruptcy_processed.xlsx' 和 'bankruptcy_processed.csv'")

# 保存Diamonds數據集
print("\n正在保存處理後的Diamonds數據集...")
df_diamonds_final.to_excel("diamonds_processed.xlsx", index=False)
df_diamonds_final.to_csv("diamonds_processed.csv", index=False)
print("Diamonds數據集已保存為 'diamonds_processed.xlsx' 和 'diamonds_processed.csv'")

print("\n所有處理後的數據集已成功保存！")



正在保存處理後的Bankruptcy數據集...
Bankruptcy數據集已保存為 'bankruptcy_processed.xlsx' 和 'bankruptcy_processed.csv'

正在保存處理後的Diamonds數據集...
Diamonds數據集已保存為 'diamonds_processed.xlsx' 和 'diamonds_processed.csv'

所有處理後的數據集已成功保存！
